<b><font size=6 color=mediumblue>Laboratory III – Part A</font></b>

## Welcome to your second OCP 5050 Python Laboratory!
In this lab you'll have a chance to further explore the ocean circulation by plotting maps and sections of ocean velocity and the streamfunction. You'll link what you see to what we've learned in class about gyre circulation and the meridional overturning circulation.

### <font color='green'>Learning Goals</font>
By the end of this notebook, you will
1. Make global plots of ocean velocity and current speed at different depths
2. Make plots of sea surface height, zonal wind stress, barotropic streamfunction, and Sverdrup transport in the North Pacific
3. Jot down some basic notes about what you see in your figures to prepare you for your lab report

---

# I) Gathering what you need

## 1. Background information
The data: we'll be using data from the [Simple Ocean Data Assimilation](https://climatedataguide.ucar.edu/climate-data/soda-simple-ocean-data-assimilation) (SODA), which is a global gridded data assimilation product that uses ocean observations to reconstruct the ocean state (temperature, salinity, velocity, etc) over the 20th (and 21st) century. The dataset is 4D--it's on a 1/2˚ longitude x 1/2˚ latitude grid with 50 depth levels. The data we are using today is a temporal subset of the total product–it is monthly output for the year 2024.

## 2. Importing things and reading in the data

Run the following block of code to import the packages you'll need to successfully complete the laboratory

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import xgcm
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cmocean.cm as cmo
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter


Below is some python code that reads in the data we'll be working with today. There are multiple variables within this dataset--you'll need to familiarize yourself with what they are by reading their metadata. One way to do this is to use ```ds['variable_name'].attrs``` to get basic information.

In [ ]:
ds=xr.open_dataset('SODA3.15.2_2024.nc')

In [ ]:
ds

The variables we are interested in today are 
1. temp
2. taux and tauy
3. u and v

Read in the data below and answer the following questions:
1. What are these variables? 
2. What are their units?
3. What are their dimensions and coordinates?

For example, if I want to know more about the zonal velocity, u, I would do the following:

In [ ]:
ds.u.attrs

In [ ]:
ds.u.dims

What are 'st_ocean', 'yu_ocean', etc? <br>What about dxt, dxu, etc?

---

# II): Making plots of the data

## 1. Ocean Velocity

### Global surface and deep maps
Make a set of filled contour plots that show the following:
1. The time mean zonal velocity at the surface ocean
2. The time mean meridional velocity at the surface ocean
3. The time mean ocean current speed at the surface
4. The time mean ocean current speed at 3500 m

Your plots must have appropriate units, a colorbar, a title, and x and y axis labels. Save the output of your plots as pdfs.

As an example, if I wanted to plot the time mean surface zonal velocity, I might do something like this:

In [ ]:
u_sfc=ds.u.sel(st_ocean=5,method='nearest').mean('time') #5m is the surface

fig,ax=plt.subplots()
ax.set_facecolor('0.5') #fills the plot background with gray so land is easier to see
cs=ax.contourf(ds.xu_ocean,ds.yu_ocean,u_sfc,
               levels=np.arange(-0.5,0.51,0.05),
               extend='both',cmap=plt.cm.RdBu_r)
cbar=plt.colorbar(cs)
cbar.set_label('u (m/s)',rotation=270,labelpad=20)
plt.title('Mean Zonal Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('surface_zonal_velocity.pdf')

In [ ]:
###YOUR CODE HERE
u_surf = ds.u.sel(st_ocean=0, method='nearest').mean('time')
v_surf = ds.v.sel(st_ocean=0, method='nearest').mean('time')
speed_surf = np.sqrt(u_surf**2 + v_surf**2)
u_deep = ds.u.sel(st_ocean=3500, method='nearest').mean('time')
v_deep = ds.v.sel(st_ocean=3500, method='nearest').mean('time')
speed_deep = np.sqrt(u_deep**2 + v_deep**2)

fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-180)})

#plot the first subplot
img1 = ax.contourf(ds.xu_ocean, ds.yu_ocean, u_surf, levels=np.arange(-0.5, 0.51, 0.05), extend='both', cmap=cmo.balance, transform=ccrs.PlateCarree())
cs1 = ax.contour(ds.xu_ocean, ds.yu_ocean, u_surf, levels=np.arange(-0.5, 0.51, 0.1), extend='both', colors='k', linewidths=0.2, transform=ccrs.PlateCarree())
cbar1 = plt.colorbar(img1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Velocity (m/s)')
cbar1.add_lines(cs1)
ax.coastlines('50m', linewidth=0.5, color='black', zorder=10)
ax.set_title('Surface Zonal Velocity')
ax.set_aspect('equal')

ax.set_xticks([-135, -90, -45, 0, 45, 90, 135], crs=ccrs.PlateCarree(central_longitude=-180))
ax.set_yticks([-90, -45, 0, 45, 90], crs=ccrs.PlateCarree(central_longitude=-180))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

fig.savefig('images/surface_zonal_velocity.png', bbox_inches='tight', pad_inches=0.05)

In [ ]:

#plot the second subplot
fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-180)})
img2 = ax.contourf(ds.xu_ocean, ds.yu_ocean, v_surf, levels=np.arange(-0.5, 0.51, 0.05), extend='both', cmap=cmo.balance, transform=ccrs.PlateCarree())
cs2 = ax.contour(ds.xu_ocean, ds.yu_ocean, v_surf, levels=np.arange(-0.5, 0.51, 0.1), extend='both', colors='k', linewidths=0.2, transform=ccrs.PlateCarree())
cbar2 = plt.colorbar(img2, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Velocity (m/s)')
cbar2.add_lines(cs2)
ax.coastlines('50m', linewidth=0.5, color='black', zorder=10)
ax.set_title('Surface Meridional Velocity')
ax.set_aspect('equal')

ax.set_xticks([-135, -90, -45, 0, 45, 90, 135], crs=ccrs.PlateCarree(central_longitude=-180))
ax.set_yticks([-90, -45, 0, 45, 90], crs=ccrs.PlateCarree(central_longitude=-180))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
fig.savefig('images/surface_meridional_velocity.png', bbox_inches='tight', pad_inches=0.05)


In [ ]:
# create the third subplot
fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300,
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-180)})
img3 = ax.contourf(ds.xu_ocean, ds.yu_ocean, speed_surf.isel(), levels=np.arange(0, 1.26, 0.05), extend='max', cmap="turbo", transform=ccrs.PlateCarree())
#cs3 = ax.contour(ds.xu_ocean, ds.yu_ocean, speed_surf.isel(), levels=np.arange(0, 1.26, 0.1), extend='max', colors='k', linewidths=0.2, transform=ccrs.PlateCarree())
cbar3 = plt.colorbar(img3, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Speed (m/s)')
#cbar3.add_lines(cs3)
ax.coastlines('50m', linewidth=0.5, color='black', zorder=10)
ax.set_title('Surface Current Speed')
ax.set_aspect('equal')

ax.set_xticks([-135, -90, -45, 0, 45, 90, 135], crs=ccrs.PlateCarree(central_longitude=-180))
ax.set_yticks([-90, -45, 0, 45, 90], crs=ccrs.PlateCarree(central_longitude=-180))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
fig.savefig('images/surface_current_speed.png', bbox_inches='tight', pad_inches=0.05)


In [ ]:
# create the fourth subplot
fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-180)})
img4 = ax.contourf(ds.xu_ocean.data, ds.yu_ocean.data, speed_deep.data, levels=np.arange(0, 0.2, 0.01), extend='max', cmap = "turbo", transform=ccrs.PlateCarree())
#cs4 = ax.contour(ds.xu_ocean.data, ds.yu_ocean.data, speed_deep.data, levels=np.arange(0, 0.2, 0.01), extend='max', colors='k', linewidths=0.2, transform=ccrs.PlateCarree())
cbar4 = plt.colorbar(img4, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Speed (m/s)')
#cbar4.add_lines(cs4)
ax.coastlines('50m', linewidth=0.5, color='black', zorder=10)
ax.set_title('Deep Current Speed')
ax.set_aspect('equal')

ax.set_xticks([-135, -90, -45, 0, 45, 90, 135], crs=ccrs.PlateCarree(central_longitude=-180))
ax.set_yticks([-90, -45, 0, 45, 90], crs=ccrs.PlateCarree(central_longitude=-180))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
fig.savefig('images/deep_current_speed.png', bbox_inches='tight', pad_inches=0.05)

#### Take a moment to look at your plots
Write down some general notes (in this notebook or anywhere else that you prefer taking notes) describing what you see. For example, where are the general areas of maximum and minimum values for each of the plots. Can you point out any major ocean currents? What are the the most obvious features in the deep ocean?

---

## 2. The North Pacific
In this part, you'll make six filled contour plots for just the North Pacific (110˚E-270˚E, 0-65˚N):
1. Plot the time mean zonal wind stress
2. Plot the time mean sea surface height
3. Plot the time mean Sverdrup transport in units of m$^{2}$ s$^{-1}$ (you can ignore Ekman effects)
4. Plot the time mean barotropic streamfunction (in Sv)
5. Plot the barotropic streamfunction (in Sv) in January and June of 2024

Recall that the streamfunction is <br>
$\psi(x,y)=\int_{xe}^{x}\int_{-H}^{0}vdzdx$

Some constants that may be helpful:<br>
$\rho_{0}$=1027 kg m$^{-3}$

$\beta$=1x10$^{-11}$ m$^{-1}$ s$^{-1}$

><font color='Red'><b>Note 1: </b></font>When you are doing a cumulative integral (an indefinite integral in x like you need to do for the streamfunction), you need to use the cumulative sum, which is `cumsum()`

As an example, here's how I'd do a cumulative zonal integral of the time mean North Pacific zonal velocity. For this integral, remember we need to multiply by dx (at velocity points) before summing:

In [ ]:
#Pick out the North Pacific domain and do the time mean
u_npac=ds.u.sel(xu_ocean=slice(110,270),yu_ocean=slice(0.5,65.5)).mean('time')
udx=u_npac*ds.dxu #multiply by dx at velocity points
u_cumulative=udx.cumsum(dim='xu_ocean') #do the cumulative sum (the integral)
#!!!!When integrating like this, the land gets replaced with zeros, so to make the land have values of nan
#again, just multiply by ds.vmask, the land mask at velocity points
u_cumulative=u_cumulative*ds.vmask

><font color='Red'><b>Note 2: </b></font> You'll need to compute the wind stress curl for one of your plots. Because I don't want to torture you (too much), run the code below when you are ready to do that computation. PLEASE BE AWARE THAT THE CODE COMPUTES THE WIND STRESS CURL GLOBALLY SO YOU NEED TO SUBSET THE WIND STRESS CURL FOR THE NORTH PACIFIC AFTER YOU COMPUTE IT

In [ ]:
grid_metrics = {('x',): ['dxt', 'dxu'], # grid cell widths in x-direction
                ('y',): ['dyt', 'dyu'], # grid cell widths in y-direction
                ('z',): ['dzt', 'dzw'], # grid cell width in z-direction     
               } 

grid=xgcm.Grid(ds, coords={'x':{'center':'xt_ocean','right':'xu_ocean'},
                           'y':{'center':'yt_ocean','right':'yu_ocean'},
                           'z':{'center':'st_ocean','right':'sw_ocean'}},
               periodic=['x'],
               autoparse_metadata=False,
               metrics=grid_metrics)

dtauydx=grid.interp(grid.derivative(ds.tauy,'x'),'y')
dtauxdy=grid.interp(grid.derivative(ds.taux,'y',boundary='extend'),'x')

#Computed wind stress curl
curltau=dtauydx-dtauxdy

xsl  = slice(110, 270)
ysl  = slice(0.5, 65.5)
rho0 = 1027.0
beta = 1e-11

curl_tau_tm = curltau.sel(xt_ocean=slice(110,270), yt_ocean=slice(0.5,65.5)).mean('time')

sv_transport = curl_tau_tm / (rho0 * beta)

fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-90)})

levsf = np.arange(-50, 51, 5)
levsc = np.arange(-50, 51, 10)

f1 = ax.contourf(sv_transport['xt_ocean'], sv_transport['yt_ocean'], sv_transport, levels=levsf, extend='both', cmap=cmo.balance, transform=ccrs.PlateCarree())
c1 = ax.contour(sv_transport['xt_ocean'], sv_transport['yt_ocean'], sv_transport, levels=levsc, colors='k', linewidths=0.4, transform=ccrs.PlateCarree())
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.80, aspect=50, fraction=0.05, label=r'Transport ($m^2 s^{-1}$)')
cbar.add_lines(c1)
ax.set_title('Sverdrup Transport')
ax.coastlines('50m', linewidth=0.5, color='black')
ax.set_xticks([-150, -120, -90, -60, -30, 0], crs=ccrs.PlateCarree(central_longitude=-90))
ax.set_yticks([15, 30, 45, 60], crs=ccrs.PlateCarree(central_longitude=-90))

lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

fig.savefig('images/sverdrup_transport.png', bbox_inches='tight', pad_inches=0.05)

In [ ]:
mean_ssh=ds.ssh.mean('time')
mean_taux = ds.taux.mean('time')
fig, ax = plt.subplots(1,1, figsize=(13,6), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-90)})
img1 = ax.contourf(ds.xu_ocean, ds.yu_ocean, mean_ssh, 
                      levels=np.arange(-1.0, 1.01, 0.05), extend='both', cmap=cmo.balance,
                      transform=ccrs.PlateCarree())
cs1 = ax.contour(ds.xu_ocean, ds.yu_ocean, mean_ssh, 
                      levels=np.arange(-1.0, 1.01, 0.1), extend='both', colors='k', linewidths=0.2,
                      transform=ccrs.PlateCarree())
cbar1 = plt.colorbar(img1, ax=ax, orientation='horizontal', pad=0.10, shrink=0.75, aspect=50, fraction=0.05, label='SSH (m)')
cbar1.add_lines(cs1)

ax.set_xticks([-150, -90, -30, 30, 90, 150], crs=ccrs.PlateCarree(central_longitude=-90))
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree(central_longitude=-90))

lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.coastlines('110m', linewidth=0.5, color='black', zorder=10)
ax.set_title('Mean Sea Surface Height')
ax.set_aspect('equal')
fig.savefig('images/mean_sea_surface_height.png', bbox_inches='tight', pad_inches=0.05)

fig, ax = plt.subplots(1,1, figsize=(13,6), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-90)})
img2 = ax.contourf(ds.xu_ocean, ds.yu_ocean, mean_taux, 
                      levels=np.arange(-0.3, 0.31, 0.01), extend='both', cmap=cmo.balance,
                      transform=ccrs.PlateCarree())
cs2 = ax.contour(ds.xu_ocean, ds.yu_ocean, mean_taux, 
                      levels=np.arange(-0.3, 0.31, 0.02), extend='both', colors='k', linewidths=0.2,
                      transform=ccrs.PlateCarree())
cbar2 = plt.colorbar(img2, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, label='Wind Stress (N/m^2)')
cbar2.add_lines(cs2)
ax.set_xticks([-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150], crs=ccrs.PlateCarree(central_longitude=-90))
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree(central_longitude=-90))

lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

ax.coastlines('110m', linewidth=0.5, color='black', zorder=10)
ax.set_title('Mean Zonal Wind Stress')
ax.set_aspect('equal')
fig.savefig('images/mean_zonal_wind_stress.png', bbox_inches='tight', pad_inches=0.05)

In [ ]:
v_npac=ds.v.sel(xu_ocean=slice(110,270),yu_ocean=slice(0.5,65.5)).mean('time')
v_z = (v_npac * ds.dzt).sum('st_ocean')  # Vertical integration
psi = (v_z * ds.dxu).cumsum('xu_ocean')  # Cumulative integration
psi_Sv = psi.where(ds.vmask == 1) / 1e6  # Convert to Sv and mask land

fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-90)})

levsf = np.arange(-70, 71, 5)
levsc = np.arange(-70, 71, 10)

f1 = ax.contourf(psi_Sv['xu_ocean'], psi_Sv['yu_ocean'], psi_Sv, levels=levsf, extend='both', cmap=cmo.balance, transform=ccrs.PlateCarree())
cs1 = ax.contour(psi_Sv['xu_ocean'], psi_Sv['yu_ocean'], psi_Sv, levels=levsc, colors='k', linewidths=0.4, transform=ccrs.PlateCarree())
ax.coastlines('50m', linewidth=0.5, color='black')
ax.set_xticks([-150, -120, -90, -60, -30, 0], crs=ccrs.PlateCarree(central_longitude=-90))
ax.set_yticks([15, 30, 45, 60], crs=ccrs.PlateCarree(central_longitude=-90))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Volume Transport (Sv)')
cbar.add_lines(cs1)
ax.set_title('Mean Barotropic Streamfunction')
fig.tight_layout()
ax.set_aspect('equal')
fig.savefig('images/mean_barotropic_streamfunction.png', bbox_inches='tight', pad_inches=0.05)

In [ ]:
jan_v_npac=ds.v.sel(xu_ocean=slice(110,270),yu_ocean=slice(0.5,65.5)).isel(time=0)
janv_z = (jan_v_npac * ds.dzt).sum('st_ocean')  # Vertical integration
jan_psi = (janv_z * ds.dxu).cumsum('xu_ocean')  # Cumulative integration
jan_psi_Sv = jan_psi.where(ds.vmask == 1) / 1e6  # Convert to Sv and mask land

fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-90)})


f1 = ax.contourf(jan_psi_Sv['xu_ocean'], jan_psi_Sv['yu_ocean'], jan_psi_Sv, levels=levsf, extend='both', cmap=cmo.balance, transform=ccrs.PlateCarree())
cs1 = ax.contour(jan_psi_Sv['xu_ocean'], jan_psi_Sv['yu_ocean'], jan_psi_Sv, levels=levsc, colors='k', linewidths=0.4, transform=ccrs.PlateCarree())
ax.coastlines('50m', linewidth=0.5, color='black')
ax.set_xticks([-150, -120, -90, -60, -30, 0], crs=ccrs.PlateCarree(central_longitude=-90))
ax.set_yticks([15, 30, 45, 60], crs=ccrs.PlateCarree(central_longitude=-90))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Volume Transport (Sv)')
cbar.add_lines(cs1)
ax.set_title('January Barotropic Streamfunction')
fig.tight_layout()
ax.set_aspect('equal')
fig.savefig('images/january_barotropic_streamfunction.png', bbox_inches='tight', pad_inches=0.05)

In [ ]:
june_v_npac=ds.v.sel(xu_ocean=slice(110,270),yu_ocean=slice(0.5,65.5)).isel(time=6)
junev_z = (june_v_npac * ds.dzt).sum('st_ocean')  # Vertical integration
june_psi = (junev_z * ds.dxu).cumsum('xu_ocean')  # Cumulative integration
june_psi_Sv = june_psi.where(ds.vmask == 1) / 1e6  # Convert to Sv and mask land

fig, ax = plt.subplots(1,1, 
    figsize=(12,5), dpi=300, sharex=True, sharey=True, 
    subplot_kw={'facecolor': '0.5', 'projection': ccrs.PlateCarree(central_longitude=-90)})

f1 = ax.contourf(june_psi_Sv['xu_ocean'], june_psi_Sv['yu_ocean'], june_psi_Sv, levels=levsf, extend='both', cmap=cmo.balance, transform=ccrs.PlateCarree())
cs1 = ax.contour(june_psi_Sv['xu_ocean'], june_psi_Sv['yu_ocean'], june_psi_Sv, levels=levsc, colors='k', linewidths=0.4, transform=ccrs.PlateCarree())
ax.coastlines('50m', linewidth=0.5, color='black')
ax.set_xticks([-150, -120, -90, -60, -30, 0], crs=ccrs.PlateCarree(central_longitude=-90))
ax.set_yticks([15, 30, 45, 60], crs=ccrs.PlateCarree(central_longitude=-90))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = plt.colorbar(f1, ax=ax, orientation='horizontal', pad=0.1, shrink=0.75, aspect=50, fraction=0.05, label='Volume Transport (Sv)')
cbar.add_lines(cs1)
ax.set_title('June Barotropic Streamfunction')
fig.tight_layout()
ax.set_aspect('equal')
fig.savefig('images/june_barotropic_streamfunction.png', bbox_inches='tight', pad_inches=0.05)

#### Take a moment to look at your plots
Write down some general notes describing what you see. Where do you expect to find regions of convergence and divergence? Upwelling and downwelling? Does the direction of your Sverdrup transport agree with this? What about your Sea Surface Height plot? How does your streamfunction plot change between January and June?

---